# Projeto Jogo Flappy Bird

Neste projeto, houve a implementação de uma técnica simples de Machine Learning para aprender a jogar o jogo Flappy Bird, o desenvolvimento do projeto contou com a criação de uma réplica do jogo, seguido da criação do algoritmo de aprendizado e, por fim, a implementação do algoritmo com o jogo. Os processos serão descritos abaixo:

In [2]:
# Importanto a biblioteca pygames para auxiliar na criação do jogo

import pygame 

# Importanto a biblioteca os para haver interação entre o código e os arquivos na pasta do computador

import os

# Importando a biblioteca random para geração de números aleatórios

import random

TELA_ALTURA = 800
TELA_LARGURA = 500

IMAGEM_CANO = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs", "pipe.png")))
IMAGEM_CHAO = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs", "base.png")))
IMAGEM_BACKGROUND = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs", "bg.png")))
IMAGENS_PASSARO = [ 
    pygame.transform.scale2x(pygame.image.load(os.path.join("imgs", "bird1.png"))),
    pygame.transform.scale2x(pygame.image.load(os.path.join("imgs", "bird2.png"))),
    pygame.transform.scale2x(pygame.image.load(os.path.join("imgs", "bird3.png")))
]

# Utilizamos o comando "pygame.transform.scale2x(), para aumentar a imagem relacionada em 2x e o comando "os.path.join()", para carregar o caminho da imagem escolhida"

pygame.font.init() 
FONTE_PONTOS = pygame.font.SysFont("arial", 50)

#Criação dos Objetos

class Passaro:
    IMGS = IMAGENS_PASSARO

    #Condições iniciais do pássaro

    ROTACAO_MAXIMA = 25
    VELOCIDADE_ROTACAO = 20
    TEMPO_ANIMACAO = 5

    def __init__(self, x, y):

        self.x = x
        self.y = y
        self.angulo = 0
        self.velocidade = 0
        self.altura = self.y
        self.tempo = 0 
        self.contagem_imagem = 0
        self.imagem = self.IMGS[0]

    # Condições do pulo do pássaro

    def pular(self):

        self.velocidade = -10.5
        self.tempo = 0
        self.altura = self.y

    # Movimentação do pássaro

    def mover(self):

        self.tempo += 1
        deslocamento = 1.5 * (self.tempo**2) + self.velocidade * self.tempo

        if deslocamento < 16:
            deslocamente = 16
        elif deslocamento < 0:
            deslocamento -= 2
        
        self.y += deslocamento

        if deslocamento < 0 or self.y < (self.altura + 50):
            if self.angulo < self.ROTACAO_MAXIMA:
                self.angulo = self.ROTACAO_MAXIMA
            else:
                if self.angulo < -90:
                    self.angulo -= self.VELOCIDADE_ROTACAO

    # Cenas do pássaro                
  
    def desenhar(self, tela):

        self.contagem_imagem += 1
        a = 0
        count = 0

        while a == 0:

            if self.contagem_imagem < self.tempo*(count + 1):
                self.imagem = self.IMGS[count]
                a += 1
            elif count > 4:
                self.imagem = self.IMGS[0]
                self.contagem_imagem = 0
                a += 1

            count += 1

        if self.angulo < -80:
            self.imagem = self.IMGS[1]
            self.contagem_imagem = self.TEMPO_ANIMACAO*2

        imagem_rotacionada = pygame.transform.rotate(self.imagem, self.angulo)
        pos_centro_imagem = self.imagem.get_rect(topleft=(self.x, self.y)).center
        retangulo = imagem_rotacionada.get_rect(center = pos_centro_imagem)
        tela.blit = (imagem_rotacionada, retangulo.topleft)

    def get_mask(self):

        return pygame.mask.from_surface(self.imagem)    

class Cano:

    DISTANCIA = 200
    VELOCIDADE = 5

    def __init__ (self, x):

        self.x = x
        self.altura = 0
        self.pos_top = 0
        self.pos_base = 0
        self.CANO_TOPO = pygame.transform.flip(IMAGEM_CANO, False, True)
        self.CANO_BASE = IMAGEM_CANO
        self.passou = False
        self.definir_altura()

    def definir_altura(self):   

        self.altura = random.randrange(50, 450)
        self.pos_top = self.altura - self.CANO_TOPO.get_height()
        self.pos_base = self.altura + self.DISTANCIA

    def mover(self):

        self.x -= self.VELOCIDADE

    def desenhar(self, tela):

        tela.blit(self.CANO_TOPO, (self.x, self.pos_top))
        tela.blit(self.CANO_BASE, (self.x, self.pos_base))
            
    def colidir(self, passaro):

        passaro_mask = passaro.get_mask()
        topo_mask = pygame.mask.from_surface(self.CANO_TOPO)
        base_mask = pygame.mask.from_surface(self.CANO_BASE)

        distancia_topo = (self.x - passaro.x, self.pos_top - round(passaro.y))
        distancia_base = (self.x - passaro.x, self.pos_base - round(passaro.y))

        colisao_topo = passaro_mask.overlap(base_mask, distancia_topo)    
        colisao_base = passaro_mask.overlap(base_mask, distancia_base)

        if colisao_topo or colisao_base:
            return True
        else:
            return False

class Chao:

    VELOCIDADE = 5
    LARGURA = IMAGEM_CHAO.get_width()
    IMAGEM =  IMAGEM_CHAO

    def __init__(self, y):

        self.y = y
        self.x1 = 0
        self.x2 = self.LARGURA

    def mover(self, y):

        self.x1 -= self.VELOCIDADE
        self.x2 -= self.VELOCIDADE

        if self.x1 + self.LARGURA < 0:
            self.x1 = self.x2 + self.LARGURA
        if self.x2 + self.LARGURA < 0:
            self.x2 = self.x2 + self.LARGURA

    def desenhar(self, tela):
        tela.blit(self.IMAGEM, (self.x1, self.y ))
        tela.blit(self.IMAGEM, (self.x2, self.y ))
    
def desenhar_tela(tela, passaros, canos, chao, pontos):

    tela.blit(IMAGEM_BACKGROUND, (0,0))

    for passaro in passaros:
        passaro.desenhar(tela)

    for cano in canos:
        cano.desenhar(tela)

    texto = FONTE_PONTOS.render(f"Pontuação: {pontos}", 1, (255, 255, 255))
    tela.blit(texto, (TELA_LARGURA - 10 - texto.get_width()), 10 )

    pygame.display.update()


# Função principal do jogo    

def main():

    passaros = [Passaro(230, 350)]
    chao = Chao(730)
    canos = [Cano(700)]
    tela = pygame.display.set_mode((TELA_LARGURA, TELA_ALTURA))
    pontos = 0
    relogio = pygame.time.Clock()

    rodando = True

    while rodando:
        relogio.tick(10)

        for evento in pygame.event.get():
            if evento.type == pygame.QUIT():
                rodando = False
                pygame.quit()
                quit()
            if evento.type == pygame.KEYDOWN:
                if evento.key == pygame.K_SPACE:
                    for passaro in passaros:
                        passaro.pular()
                    
        for passaro in passaros:
            passaro.mover()

        chao.mover()

        adicionar_cano = False
        remover_canos = []

        for cano in canos:
            for i, passaro in enumerate(passaros):
                if cano.colidir(passaro):
                    passaros.pop(i)
                if not cano.passou() and passaro.x > cano.x:
                    cano.passou = True
                    adicionar_cano = True
            
            cano.mover()
            if cano.x + cano.CANO_TOPO.get_width() < 0:
                remover_canos.append(cano)

        if adicionar_cano: 
            pontos += 1
            canos.append(Cano(600))

        for cano in remover_canos:
            canos.remove(cano)

        for i, passaro in enumerate(passaros):
            if (passaro.y + passaro.imagem.get_height() > chao.y or passaro.y < 0):
                passaros.pop(i)
        
        desenhar_tela(tela, passaros, canos, chao, pontos)

if __name__ == "__main__" :
    main()



        



        


    




        

TypeError: 'int' object is not callable

: 